# **Import Libraries**

In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import random
from collections import deque
from tqdm import tqdm
import matplotlib.pyplot as plt
from ipywidgets import Output
import sys

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("GPU is available")
    print(physical_devices)
else:
    print("GPU is not available")

GPU is available
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **Frogger Enviroment**

In [3]:
# create env with gymnasium (use ram or rgb state)
env = gym.make(
    "ALE/Frogger-ram-v5", # "ALE/Frogger-ram-v5" or "ALE/Frogger-v5"
    render_mode="rgb_array", # rgb_array or human
    difficulty = 0, # [0, 1]
    mode = 0 # [0, 1, 2]
    ) 

print(f"State Frame Size: {env.observation_space}")
print(f"Number Of Actions: {env.action_space.n}")

actions_space = possible_actions = np.array(np.identity(env.action_space.n,dtype=int).tolist())
print(f"Possible Actions: \n {actions_space}")

env.reset()
observation = env.step(1)
print(f"Obervation: \n {observation}")
print(observation[0].shape)

State Frame Size: Box(0, 255, (128,), uint8)
Number Of Actions: 5
Possible Actions: 
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
Obervation: 
 (array([143,  20,   5,  40,   6,  45, 147, 127,  28,  33,  39,  63, 100,
        85, 120,  86,  87,  52, 135,  60, 133, 119, 252,  36,  35, 213,
        19, 118, 188,  11, 164,  69, 229,  71, 217, 216, 123, 200, 184,
         6, 124, 119, 171, 154, 255,  91,  91, 195,  80,   0,   0,   4,
        40,   0,   0,   0, 234, 254, 234, 254,  29, 254,  51, 254, 157,
       254, 157, 254,  84, 254,   1,   0, 115, 115, 149,   0,   0,   0,
         0,   0,   4,   4,   1,   1,   0, 253, 155, 253, 255,  11,   0,
        75,   0,   1,   8,  63,  30,   0,   0,   0,   0,   0,   0,   0,
        46, 253,   0, 115, 255, 198,   1, 255,   0, 163,  58,  80, 255,
        80, 255,  28,  12,   0,   0,  91,   0,   0, 124, 250], dtype=uint8), 0.0, False, False, {'lives': 4, 'episode_frame_number': 4, 'frame_number': 4})
(128,)


# **Hyperparameter**

In [4]:
env = gym.make("ALE/Frogger-ram-v5", render_mode="rgb_array", difficulty = 0, mode = 0) 

# MODEL Hyperparameters
STATE_SIZE = env.observation_space.shape[0]
ACTIONS_SIZE = env.action_space.n
ACTIONS = list(range(0, ACTIONS_SIZE))
LEARNING_RATE = 0.01 # Learning Rate (alpha)

# AGENT Hyperparameters (epsilon greedy strategy)
EPSILON = 1.0
EPSILON_MIN = 0.01 # EPSELON value where exploreation stops
EPSILON_DECAY_RATE = 0.995 # the higher the longer the exploreation takes (Linear Decay: EPSELON * DECAY_RATE)
GAMMA = 0.99 # Discounting rate (lower -> agent thinks more long term)

# TRAINING Hyperparameters
RENDER_INTERVALL = 50 # Intervall when the game is rendered
TOTAL_EPISODES = 1000000
MINI_BATCHES_REPLAY = 64
REPLAY_BUFFER_MEMORY = 10000
STACKED_FRAMES_SIZE = 4
AVERAGE_WINDOW = 50

# **Deep Q-Learning Neural Network Model**

In [5]:
"""
def build_dqn(input_shape=(STATE_SIZE*STACKED_FRAMES_SIZE,)):
    model = models.Sequential()
    model.add(layers.Dense(24, input_shape=input_shape, activation='relu'))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(ACTIONS_SIZE, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
    return model

"""


def build_dqn(input_shape=(STATE_SIZE*STACKED_FRAMES_SIZE,)):
    model = models.Sequential()
    model.add(layers.Reshape((16, 32), input_shape=input_shape))  # Reshape to (16, 32)
    model.add(layers.Conv1D(32, 3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(4))  # Upsample to (64, 32)
    model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(2))  # Upsample to (128, 64)
    model.add(layers.Conv1D(4, 3, activation='linear', padding='same'))  # Output (128, 4)
    model.add(layers.Reshape((128, 4)))  # Reshape to (128, 4)
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
    return model

# **Stacked Frames**

In [6]:
STACKED_FRAMES = deque([np.zeros((STATE_SIZE), dtype=np.int32) for i in range(STACKED_FRAMES_SIZE)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):

    if is_new_episode:
        # clear stack for new episode
        stacked_frames = deque([np.zeros((STATE_SIZE), dtype=np.int32) for i in range(STACKED_FRAMES_SIZE)], maxlen=4)
        
        # Add the same frame 4 times to the deque since its a new episode
        stacked_frames.append(state)
        stacked_frames.append(state)
        stacked_frames.append(state)
        stacked_frames.append(state)
        
        # Stack the frames with numpy (join all 4 frames)
        stacked_frames_array = np.stack(stacked_frames, axis=0)

    elif not is_new_episode:
        # append new frame and remove oldest frame
        stacked_frames.append(state)

        # Stack the frames with numpy (join all 4 frames)
        stacked_frames_array = np.stack(stacked_frames, axis=0) 

    return stacked_frames_array, stacked_frames

# **Replay Buffer**

In [7]:
class ReplayBuffer:
    def __init__(self):
        # deque that ther are only max REPLAY_BUFFER_MEMORY items in the list
        # deque = remove oldest item
        self.buffer = deque(maxlen=REPLAY_BUFFER_MEMORY)
    
    def add(self, experience):
        # add item to buffer
        self.buffer.append(experience)
    
    def sample(self):
        return random.sample(self.buffer, MINI_BATCHES_REPLAY) # 16 (MINI_BATCHES_REPLAY) samples to retrain the mdoel

# **DQN Agent**

In [8]:
class DQNAgent:
    def __init__(self):
        self.memory = ReplayBuffer()
        self.EPSILON = EPSILON
        self.model = build_dqn()
        self.target_model = build_dqn()
        self.update_target_model()
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        flattend_array_state = np.expand_dims(state.reshape(-1), axis=0)
        flattend_array_next_state = np.expand_dims(next_state.reshape(-1), axis=0)
        self.memory.add((flattend_array_state, action, reward, flattend_array_next_state, done))
    
    def predict_action(self, stacked_array):
        # flatten array for model
        flattend_array = np.expand_dims(stacked_array.reshape(-1), axis=0)

        if np.random.rand() <= EPSILON:
            return random.randrange(ACTIONS_SIZE)
        q_values = self.model.predict(flattend_array)
        return np.argmax(q_values[0])
    
    def replay(self):
        print("Replay")
        minibatch = self.memory.sample()
        i = 0
        for state, action, reward, next_state, done in tqdm(minibatch):
            i += 1
            # Predict Target Q-Values
            target = self.model.predict(state, verbose=0)
            if done:
                # If the episode is done the target Q-value for the taken action is set to the received reward
                target[0][action] = reward
            elif not done:
                # If the episode is not done, the target Q-value for the taken action is updated using the Bellman equation
                t = self.target_model.predict(next_state, verbose=0)[0]
                target[0][action] = reward + GAMMA * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
                
        if self.EPSILON > EPSILON_MIN:
            self.EPSILON *= EPSILON_DECAY_RATE

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

# **Reward Graph**

In [9]:
%matplotlib qt
average_rewards = []
rewards_per_episode = []

plt.figure(figsize=(12, 6))
reward_line, = plt.plot(rewards_per_episode, label='Reward per Episode')
average_line, = plt.plot(average_rewards, label='Moving Average Reward')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Episode Reward and Moving Average Reward over Time')
plt.legend()

# Display plot in a new window
plt.show()

# **Training**

In [10]:
%matplotlib qt

agent = DQNAgent()

for episode in range(TOTAL_EPISODES):
    # create env (human to render game and see actions)
    if episode % RENDER_INTERVALL == 0 and episode != 0:
        env = gym.make("ALE/Frogger-ram-v5", render_mode="human", difficulty = 0, mode = 0) 
    else:
        env = gym.make("ALE/Frogger-ram-v5", render_mode="rgb_array", difficulty = 0, mode = 0) 
    state = env.reset()[0]

    # reset episode variables
    done = False
    total_reward = 0
    step_count = 0
    is_new_episode = True

    # initilize episode stack
    stacked_array_state, STACKED_FRAMES = stack_frames(STACKED_FRAMES, state, is_new_episode)
    
    while not done:
        # update step_count
        step_count += 1

        # Predict action
        if is_new_episode and step_count <= 110:
            # jsut wait for first 100 steps because of initialization 
            action = 0
            next_state, reward, done, loss_of_live, info = env.step(0)
        else:
            action = agent.predict_action(stacked_array_state)
            next_state, reward, done, loss_of_live, info = env.step(action)
            is_new_episode = False

        # Predict action
        action = agent.predict_action(stacked_array_state)
        next_state, reward, done, loss_of_live, info = env.step(action)
        
        # Do Stacking
        stacked_array_next_state, STACKED_FRAMES = stack_frames(STACKED_FRAMES, next_state, is_new_episode)

        # update reward
        total_reward += reward

        # store action infromation in memory
        next_state = np.reshape(next_state, [1, STATE_SIZE])
        agent.remember(stacked_array_state, action, reward, stacked_array_next_state, done)

        # set the next state to the current state
        stacked_array_state = stacked_array_next_state
        
        # if game finished (won or lose)
        if done:
            agent.update_target_model()

            rewards_per_episode.append(total_reward)

            if len(rewards_per_episode) >= AVERAGE_WINDOW:
                moving_average = np.mean(rewards_per_episode[-AVERAGE_WINDOW:])
            else:
                moving_average = np.mean(rewards_per_episode)
            average_rewards.append(moving_average)

            # reward graph
            reward_line.set_data(range(episode + 1), rewards_per_episode)
            average_line.set_data(range(episode + 1), average_rewards)
            plt.xlim(0, episode + 1)
            plt.ylim(0, max(max(rewards_per_episode), max(average_rewards)) + 10)
            plt.pause(0.01)

            print(f"Episode: {episode}/{TOTAL_EPISODES}, Total Reward: {total_reward}, Moving AVG. Reward: {moving_average}, Steps: {step_count}, Epsilon: {agent.EPSILON:.2}")
            break

    if episode % 4 == 0 and episode != 0:
        agent.replay()

Episode: 0/1000000, Total Reward: 5.0, Moving AVG. Reward: 5.0, Steps: 122, Epsilon: 1.0
Episode: 1/1000000, Total Reward: 4.0, Moving AVG. Reward: 4.5, Steps: 195, Epsilon: 1.0
Episode: 2/1000000, Total Reward: 10.0, Moving AVG. Reward: 6.333333333333333, Steps: 145, Epsilon: 1.0
Episode: 3/1000000, Total Reward: 6.0, Moving AVG. Reward: 6.25, Steps: 153, Epsilon: 1.0
Episode: 4/1000000, Total Reward: 9.0, Moving AVG. Reward: 6.8, Steps: 124, Epsilon: 1.0
Replay


100%|██████████| 64/64 [00:11<00:00,  5.68it/s]


Episode: 5/1000000, Total Reward: 8.0, Moving AVG. Reward: 7.0, Steps: 145, Epsilon: 0.99
Episode: 6/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.0, Steps: 134, Epsilon: 0.99
Episode: 7/1000000, Total Reward: 9.0, Moving AVG. Reward: 7.25, Steps: 159, Epsilon: 0.99
Episode: 8/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.222222222222222, Steps: 172, Epsilon: 0.99
Replay


100%|██████████| 64/64 [00:09<00:00,  7.08it/s]


Episode: 9/1000000, Total Reward: 8.0, Moving AVG. Reward: 7.3, Steps: 194, Epsilon: 0.99
Episode: 10/1000000, Total Reward: 8.0, Moving AVG. Reward: 7.363636363636363, Steps: 138, Epsilon: 0.99
Episode: 11/1000000, Total Reward: 9.0, Moving AVG. Reward: 7.5, Steps: 133, Epsilon: 0.99
Episode: 12/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.461538461538462, Steps: 145, Epsilon: 0.99
Replay


...

100%|██████████| 64/64 [00:08<00:00,  7.37it/s]


Episode: 49/1000000, Total Reward: 5.0, Moving AVG. Reward: 7.34, Steps: 145, Epsilon: 0.94
Episode: 50/1000000, Total Reward: 6.0, Moving AVG. Reward: 7.36, Steps: 127, Epsilon: 0.94
Episode: 51/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.42, Steps: 139, Epsilon: 0.94
Episode: 52/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.36, Steps: 176, Epsilon: 0.94
Replay


100%|██████████| 64/64 [00:08<00:00,  7.12it/s]


Episode: 53/1000000, Total Reward: 4.0, Moving AVG. Reward: 7.32, Steps: 145, Epsilon: 0.94
Episode: 54/1000000, Total Reward: 9.0, Moving AVG. Reward: 7.32, Steps: 115, Epsilon: 0.94
Episode: 55/1000000, Total Reward: 5.0, Moving AVG. Reward: 7.26, Steps: 125, Epsilon: 0.94
Episode: 56/1000000, Total Reward: 7.0, Moving AVG. Reward: 7.26, Steps: 141, Epsilon: 0.94
Replay


 20%|██        | 13/64 [00:01<00:06,  7.43it/s]